In [1]:
import praw
import json
import csv
import requests
from datetime import date
from textblob import TextBlob, Word

In [2]:
# pull security info from json file

def create_reddit_object(json_file = "reddit-login.json", json_key = "comment-crawler-login"):
    with open(json_file) as f:
        data = json.load(f)
        session = requests.Session()
        session.verify = 'C:\\Users\\p40014d\\OneDrive - AholdDelhaize.com\\Documents\\Github Code\\Reddit-Web-Scraper\\certReddit.cer'

        user_info = data[json_key][0]
        reddit = praw.Reddit(client_id = user_info['client_id'],
                            client_secret = user_info['client_secret'],
                            user_agent = user_info['user_agent'],
                            username = user_info['username'],
                            password = user_info['password'],
                            requestor_kwargs={'session': session})

    return reddit
    


In [3]:
# DATA PULL

reddit = create_reddit_object()
post_id = input("Enter the ID from the URL:").strip()
today = date.today().strftime('%Y-%m-%d')
submission = reddit.submission(id=post_id)
subred_name = submission.subreddit.display_name
new_csv_file_name = f'C:/Users/p40014d/OneDrive - AholdDelhaize.com/Documents/Web Scraping/Self-Checkout - Reddit/{subred_name} Source ID {post_id} Scrape Results {today}.csv'

#prepare CSV file
print("Creating new CSV file...")
try:
    with open(new_csv_file_name, 'w', encoding = 'utf_8_sig', newline = '') as csv_file:
        csv_write = csv.writer(csv_file)
        csv_write.writerow(['Comment ID', 'UTC DateTime', 'Comment', 'Score', 'Polarity', 'Subjectivity', 'Username', 'User Karma (total)', 'Parent Comment ID' ])

        #pull initial post's title, author, date, score
        og_post_sentiment = TextBlob(submission.title)
        print("Scraping Reddit comments...")
        csv_write.writerow([post_id, submission.created, submission.title, submission.score, og_post_sentiment.sentiment.polarity, og_post_sentiment.sentiment.subjectivity, submission.author, "", "Original Post"])

        # loop through comments
        submission.comments.replace_more(limit=2)
        for comment in submission.comments.list():
            
            # Enter null values for [deleted] items
            try:
                username = comment.author.name
            except:
                username = ""
                
            try:
                comment_score = comment.score
            except:
                comment_score = 0
            
            # this step is the time-waster here! takes several minutes
            try:
                users_score = comment.author.comment_karma
            except:
                users_score = 0

            try:
                comment_body = str(comment.body)
            except:
                comment_body = ""

            # try to write a CSV file with new data
            comment_sent = TextBlob(comment_body)
            try:
                csv_write.writerow([comment.id,
                                    comment.created,
                                    comment_body,
                                    comment_score,
                                    comment_sent.polarity,
                                    comment_sent.subjectivity,
                                    username,
                                    users_score,
                                    comment.parent_id]) #needs brackets to keep strings combined
            except PermissionError as err:
                print('Close all instances of the file open.')
            except AttributeError as err:
                print(err)
    print("Reddit comments successfully extracted!")
except PermissionError as err:
    print('Close all instances of the file open.')

Creating new CSV file...
Scraping Reddit comments...
Reddit comments successfully extracted!
